# Mock Hackathon

In [136]:
!pip install category_encoders
import category_encoders as ce
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve,
    average_precision_score, confusion_matrix
)
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score

sample submission

In [137]:
local_path = os.path.join('data','sample_submission.csv')
url = 'https://raw.githubusercontent.com/samsung-ai-course/8th-9th-edition/refs/heads/main/Chapter%205%20-%20Responsible%20AI/data/sample_submission.csv'
df_sample_submission = pd.read_csv(url)
df_sample_submission

,id,proba
0,7991,0.5
1,31470,0.5
2,27345,0.5
3,7509,0.5
4,61999,0.5
...,...,...
4495,87722,0.5
4496,16925,0.5
4497,35300,0.5
4498,86129,0.5


test data to create submissions on

In [138]:
local_path = os.path.join('data','test.csv')
url = 'https://raw.githubusercontent.com/samsung-ai-course/8th-9th-edition/refs/heads/main/Chapter%205%20-%20Responsible%20AI/data/test.csv'
df_test = pd.read_csv(url, low_memory=False)
df_test

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x44d1,CUS_0xb1da,August,Paulj,35,671-03-8487,Doctor,110721.66_,9259.805000,3,...,6.0,_,281.42,28.620300,26 Years and 6 Months,No,209.245449,131.5562784570291,High_spent_Medium_value_payments,835.178773
1,0xce64,CUS_0x6f55,July,Leongr,53,923-48-1651,Writer,18579.36,1298.280000,0,...,1.0,Good,1329.98,29.681948,29 Years and 9 Months,No,16.174857,150.7169895127531,!@9#%8,252.936153
2,0xb63b,CUS_0x880b,February,Jones Lisag,21,790-45-5982,Writer,7830.03,775.502500,8,...,8.0,Bad,1255.57,27.130360,5 Years and 5 Months,Yes,31.593600,47.532261487489116,Low_spent_Medium_value_payments,278.424389
3,0x41ff,CUS_0x4a83,June,NaN,19,408-42-1764,Engineer,35322.24_,2805.520000,5,...,2.0,_,907.76,34.294300,30 Years and 6 Months,NM,61.725788,185.52382946164252,!@9#%8,293.302383
4,0x18145,CUS_0x7d6c,August,Eileenc,24,424-12-8828,Scientist,17307708.0,7093.165000,5,...,4.0,Standard,218.44,39.448697,28 Years and 0 Months,No,72.633969,371.05530058370016,High_spent_Small_value_payments,525.627230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,0x21800,CUS_0x4bef,March,McAllisterb,18,783-63-1238,Journalist,38505.32,2922.147520,7,...,4.0,Standard,2159.19,25.539420,10 Years and 1 Months,Yes,214.658697,24.791647923065504,High_spent_Large_value_payments,450.756468
4496,0x792b,CUS_0xa6f9,June,Kevin Drawbaughf,29,#F%$D@*&8,Scientist,46722.72,3781.560000,9,...,16.0,_,4764.13,28.066109,8 Years and 10 Months,Yes,284.659751,279.81362006108316,Low_spent_Small_value_payments,103.682629
4497,0xe4d6,CUS_0x262e,May,Karina Grazinax,23,216-94-2190,Journalist,8983.565,645.630417,6,...,8.0,Standard,1270.65,28.235121,15 Years and 0 Months,Yes,9.885125,16.832254838086712,Low_spent_Medium_value_payments,317.845662
4498,0x20eab,CUS_0x3ce2,February,Suzannek,29,776-82-8001,Engineer,14095.93,1310.660833,9,...,7.0,Bad,2790.05,27.949570,1 Years and 6 Months,Yes,37.745141,54.5238056664076,High_spent_Small_value_payments,298.797136


historical data to train models on

In [139]:
local_path = os.path.join('data','history.csv')
url = 'https://raw.githubusercontent.com/samsung-ai-course/8th-9th-edition/refs/heads/main/Chapter%205%20-%20Responsible%20AI/data/history.csv'
df_train = pd.read_csv(url, low_memory=False)
print(df_train.shape)
df_train.head(3)

(33412, 28)


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
2,0x1607,CUS_0xd40,June,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,27.262259,22 Years and 6 Months,No,49.574949,62.430172331195294,!@9#%8,340.4792117872438,Good


# Exercise 1

Train a model on this data and perform threshold analysis. Create a submission that maximizes for f1_score

In [140]:
df_train.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,28328.000000,33412.000000,33412.000000,33412.000000,33412.000000,32763.000000,33412.000000,33412.000000
mean,3849.384660,17.824823,23.432030,75.723961,25.335867,29.462687,32.188057,1402.643999
std,3054.153183,118.492679,131.333891,466.159560,16.871284,195.399167,5.103672,8291.358715
min,319.556250,-1.000000,0.000000,1.000000,-5.000000,0.000000,20.257073,0.000000
25%,1485.806667,4.000000,5.000000,9.000000,12.000000,4.000000,27.958615,36.347867
50%,2937.987083,6.000000,6.000000,18.000000,22.000000,7.000000,32.218877,75.224655
75%,5280.560402,8.000000,8.000000,26.000000,37.000000,10.000000,36.410374,169.797237
max,15204.633333,1794.000000,1499.000000,5774.000000,67.000000,2583.000000,48.247003,82331.000000


In [141]:
df_train.dtypes

,0
ID,object
Customer_ID,object
Month,object
Name,object
Age,object
SSN,object
Occupation,object
Annual_Income,object
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,int64


In [142]:
num_cols_str = [
    'Age','Annual_Income','Num_of_Loan','Num_of_Delayed_Payment',
    'Changed_Credit_Limit','Outstanding_Debt','Credit_History_Age',
    'Amount_invested_monthly','Monthly_Balance'
]

for col in num_cols_str:
    df_train[col] = (
        df_train[col].astype(str)
        .str.replace(r'[^0-9.]', '', regex=True)
        .replace('', '0')  # opcional: se string ficar vazia, troca por 0
        .astype(float)
    )


df_train['Monthly_Inhand_Salary'] = df_train['Monthly_Inhand_Salary'].fillna(
    df_train['Annual_Income'] / 12
)

#moda_embarque = df_clean['Embarked'].mode()
#df_clean['Embarked']=df_clean['Embarked'].fillna(moda_embarque.values[0])

In [143]:
# train test split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
# 1️⃣ Separar X e y
X = df_train.drop(columns=["ID", "Customer_ID", "Name", "SSN", "Credit_Score",])
y = df_train["Credit_Score"]

# 2️⃣ Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3️⃣ Train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

# 4️⃣ OneHotEncoder só nas categóricas
num_cols = X_train.select_dtypes(include=['float64','int64']).columns
num_imputer = SimpleImputer(strategy='mean')
X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
X_val[num_cols]   = num_imputer.transform(X_val[num_cols])
X_test[num_cols]  = num_imputer.transform(X_test[num_cols])

# Colunas categóricas
cat_cols = X_train.select_dtypes(include=['object']).columns
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train[cat_cols] = cat_imputer.fit_transform(X_train[cat_cols])
X_val[cat_cols]   = cat_imputer.transform(X_val[cat_cols])
X_test[cat_cols]  = cat_imputer.transform(X_test[cat_cols])

cat_cols = [
    'Credit_Mix',
    'Payment_of_Min_Amount',
    'Payment_Behaviour',
    'Occupation',
    'Month',
    'Type_of_Loan'
]

ohe = ce.OneHotEncoder(cols=cat_cols)

X_train = ohe.fit_transform(X_train)
X_val   = ohe.transform(X_val)
X_test  = ohe.transform(X_test)



In [144]:
regressao = LogisticRegression()
regressao.fit(X_train, y_train)

LogisticRegression()

In [145]:
preds = regressao.predict(X_val)
accuracy_score = lambda y_true, preds: sum(preds == y_true) / len(y_true)

In [146]:
accuracy_score(y_val, preds)

0.764122708567153

In [147]:


f1 = f1_score(y_val, preds, average='weighted')  # weighted se tiver múltiplas classes
print(f1)

0.6619534014413785


Melhor F1: 0.38171636143484183 com threshold = 0.1


# Exercise 2

Assume the cost of a False positive is 30€ and the benefit of a True Positive is 70€, create a submission that maximizes for this.

# Exercise 3

Evaluate the fairness of your model using [aequitas](https://github.com/dssg/aequitas)